In [22]:
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
import numpy as np
from numpy.linalg import norm
import os
from tqdm.notebook import tqdm
import pickle
import cv2
#from sklearn.neighbors import NearestNeighbors
import faiss

import warnings
warnings.filterwarnings('ignore')

In [23]:
features = np.array(pickle.load(open('features.pkl','rb')))

In [24]:
features = np.array(features).astype('float32')

In [25]:
print(features.shape)

(44441, 2048)


In [26]:
index = faiss.IndexFlatL2(features.shape[1])
index.add(features)

In [27]:
filesname = pickle.load(open('filesname.pkl','rb'))

In [28]:
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable= False

In [29]:
model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])

In [ ]:
img =image.load_img('sample\shorts.jpg',target_size=(224,224))
img_array = image.img_to_array(img).astype('float32')
expanded_img_array=np.expand_dims(img_array,axis=0)
preprocessed_img=preprocess_input(expanded_img_array)
result=model.predict(preprocessed_img).flatten()
normalized_result=result/norm(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [31]:
print(img_array.shape)

(224, 224, 3)


In [32]:
distances, indices = index.search(normalized_result.reshape(1, -1), k=5)

In [37]:


for idx in indices[0][1:6]:  # skip first one if it's the same image
    result_image = cv2.imread(filesname[idx])  # use your actual file path list
    cv2.imshow('Similar Image', cv2.resize(result_image, (512, 512)))
    cv2.waitKey(0)

In [34]:
#neighbors = NearestNeighbors(n_neighbors=6,algorithm='brute',metric='euclidean')
#neighbors.fit(features)

In [35]:
#distances , indices = neighbors.kneighbors([normalized_result])

In [36]:
#for file in indices[0][1:6]:
  # result_image = cv2.imread((filesname[file]))
  # cv2.imshow('output',cv2.resize(result_image,(512,512)))
  # cv2.waitKey(0)